<h1><center>РАБОТА С ПРИЗНАКАМИ</center></h1>

Для начала определите глобальные переменные, которые понадобятся при выполнении заданий. Используйте уже существующий эксперимент в MLflow. Вот пример кода, который можно взять в качестве шаблона для объявления переменных:

In [ ]:
import pandas as pd
import numpy as np
import os
import psycopg
import matplotlib.pyplot as plt
import seaborn as sns
import mlflow
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import ( OneHotEncoder, SplineTransformer, QuantileTransformer, 
                                    RobustScaler, PolynomialFeatures, KBinsDiscretizer )

TABLE_NAME = "clean_users_churn" # таблица с данными

TRACKING_SERVER_HOST = "127.0.0.1"
TRACKING_SERVER_PORT = 5000

EXPERIMENT_NAME = "priznaki" # название эксперимента
RUN_NAME = "preprocessing" 
REGISTRY_MODEL_NAME = "priznak" # название зарегистрированной модели 

experiment_id = mlflow.create_experiment(EXPERIMENT_NAME) 

После этого загрузите ваш набор данных из базы данных Postgres — так, как вы это делали в предыдущих уроках.
<br>Получившийся датафрейм сохраните в переменную df.

In [4]:
connection = {"sslmode": "require", "target_session_attrs": "read-write"}
postgres_credentials = {"host": 'rc1b-uh7kdmcx67eomesf.mdb.yandexcloud.net', #os.getenv("DB_DESTINATION_HOST"),
                        "port": '6432', #os.getenv("DB_DESTINATION_PORT"),
                        "dbname": 'playground_mle_20250529_05fed48463', #os.getenv("DB_DESTINATION_NAME"),
                        "user": 'mle_20250529_05fed48463', #os.getenv("DB_DESTINATION_USER"),
                        "password": '0c567edd8ad8472e87d5c85cc4d664e4' } #os.getenv("DB_DESTINATION_PASSWORD")}
connection.update(postgres_credentials)

with psycopg.connect(**connection) as conn:
    with conn.cursor() as cur:
        cur.execute(f"SELECT * FROM {TABLE_NAME}")
        data = cur.fetchall()
        columns = [col[0] for col in cur.description]

df = pd.DataFrame(data, columns=columns)
df.head(2) 

,id,customer_id,begin_date,end_date,type,paperless_billing,payment_method,monthly_charges,total_charges,internet_service,...,device_protection,tech_support,streaming_tv,streaming_movies,gender,senior_citizen,partner,dependents,multiple_lines,target
0,1,7590-VHVEG,2020-01-01,NaT,Month-to-month,Yes,Electronic check,29.85,29.85,DSL,...,No,No,No,No,Female,0,Yes,No,No,None
1,2,5575-GNVDE,2017-04-01,NaT,One year,No,Mailed check,56.95,1889.50,DSL,...,Yes,No,No,No,Male,0,No,No,No,None


Выберите команду, которая выделит нечисловые колонки вашего датасета:

In [ ]:
obj_df = df.select_dtypes(include="object")

Index(['customer_id', 'type', 'paperless_billing', 'payment_method',
       'internet_service', 'online_security', 'online_backup',
       'device_protection', 'tech_support', 'streaming_tv', 'streaming_movies',
       'gender', 'partner', 'dependents', 'multiple_lines', 'target'],
      dtype='object')

**Задание 1**

После того как вы выделили категориальные колонки, закодируйте их для подачи в вашу модель:

In [10]:
# определение категориальных колонок, которые будут преобразованы
cat_columns = ["type", "payment_method", "internet_service", "gender"]

# создание объекта OneHotEncoder для преобразования категориальных переменных
# auto - автоматическое определение категорий
# ignore - игнорировать ошибки, если встречается неизвестная категория
# max_categories - максимальное количество уникальных категорий
# sparse_output - вывод в виде разреженной матрицы, если False, то в виде обычного массива
# drop="first" - удаляет первую категорию, чтобы избежать ловушки мультиколлинеарности
encoder_oh = OneHotEncoder(categories='auto', handle_unknown='ignore', max_categories=10, sparse_output=False, drop='first') # ваш код здесь #

# применение OneHotEncoder к данным. Преобразование категориальных данных в массив
encoded_features = encoder_oh.fit_transform(df[cat_columns].to_numpy()) # ваш код здесь #

# преобразование полученных признаков в DataFrame и установка названий колонок
# get_feature_names_out() - получение имён признаков после преобразования
encoded_df = pd.DataFrame(encoded_features, columns=encoder_oh.get_feature_names_out(cat_columns)) # ваш код здесь #

# конкатенация исходного DataFrame с новым DataFrame, содержащим закодированные категориальные признаки
# axis=1 означает конкатенацию по колонкам
obj_df = pd.concat([obj_df, encoded_df], axis=1)

obj_df.head(2)

,customer_id,type,paperless_billing,payment_method,internet_service,online_security,online_backup,device_protection,tech_support,streaming_tv,...,dependents,multiple_lines,target,type_One year,type_Two year,payment_method_Credit card (automatic),payment_method_Electronic check,payment_method_Mailed check,internet_service_Fiber optic,gender_Male
0,7590-VHVEG,Month-to-month,Yes,Electronic check,DSL,No,Yes,No,No,No,...,No,No,None,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,5575-GNVDE,One year,No,Mailed check,DSL,Yes,No,Yes,No,No,...,No,No,None,1.0,0.0,0.0,0.0,1.0,0.0,1.0


Сейчас поработайте с числовыми признаками: monthly_charges и total_charges. Из них можно сгенерировать довольно много признаков для вашей модели. 

**Задание 2**

Напишите код преобразования числовых признаков в списке num_columns, используя следующие энкодеры:
- SplineTransformer,
- QuantileTransformer,
- RobustScaler,
- PolynomialFeatures,
- KBinsDiscretizer.

In [17]:
num_columns = ["monthly_charges", "total_charges"]

n_knots = 3
degree_spline = 4
n_quantiles=100
degree = 3
n_bins = 5
encode = 'ordinal'
strategy = 'uniform'
subsample = None

# num_df = df.select_dtypes(include=['number'])

num_df = df[num_columns].copy()

# SplineTransformer
encoder_spl = SplineTransformer(n_knots=n_knots, degree=degree_spline) # ваш код здесь #
encoded_features = encoder_spl.fit_transform(df[num_columns].to_numpy()) # ваш код здесь #
encoded_df = pd.DataFrame( encoded_features, columns=encoder_spl.get_feature_names_out(num_columns) )
num_df = pd.concat([num_df, encoded_df], axis=1)

# QuantileTransformer
encoder_q = QuantileTransformer(n_quantiles=n_quantiles) #, output_distribution='normal') # ваш код здесь #
encoded_features = encoder_q.fit_transform(df[num_columns].to_numpy()) # ваш код здесь #
encoded_df = pd.DataFrame(encoded_features, columns=encoder_q.get_feature_names_out(num_columns)) # ваш код здесь #
encoded_df.columns = [col + f"_q_{n_quantiles}" for col in num_columns]
num_df = pd.concat([num_df, encoded_df], axis=1)

# RobustScaler
encoder_rb = RobustScaler() # ваш код здесь #
encoded_features = encoder_rb.fit_transform(df[num_columns].to_numpy()) # ваш код здесь #
encoded_df = pd.DataFrame(encoded_features, columns=encoder_rb.get_feature_names_out(num_columns)) # ваш код здесь #
encoded_df.columns = [col + f"_robust" for col in num_columns]
num_df = pd.concat([num_df, encoded_df], axis=1)

# PolynomialFeatures
encoder_pol = PolynomialFeatures(degree=degree) # ваш код здесь #
encoded_features = encoder_pol.fit_transform(df[num_columns].to_numpy()) # ваш код здесь #
encoded_df = pd.DataFrame(encoded_features, columns=encoder_pol.get_feature_names_out(num_columns)) # ваш код здесь #
# get all columns after the intercept and original features
encoded_df.columns = encoder_pol.get_feature_names_out(num_columns)
encoded_df = encoded_df.iloc[:, 1 + len(num_columns):]
encoded_df.columns = [f"{col}_poly" for col in encoded_df.columns]

# KBinsDiscretizer
encoder_kbd = KBinsDiscretizer(n_bins=n_bins, encode=encode, strategy=strategy, subsample=subsample) # ваш код здесь #
encoded_features = encoder_kbd.fit_transform(df[num_columns].to_numpy()) # ваш код здесь #
encoded_df = pd.DataFrame(encoded_features, columns=encoder_kbd.get_feature_names_out(num_columns)) # ваш код здесь #
encoded_df.columns = [col + f"_bin" for col in num_columns]
num_df = pd.concat([num_df, encoded_df], axis=1) # ваш код здесь #

num_df.head(2)

,monthly_charges,total_charges,monthly_charges_sp_0,monthly_charges_sp_1,monthly_charges_sp_2,monthly_charges_sp_3,monthly_charges_sp_4,monthly_charges_sp_5,total_charges_sp_0,total_charges_sp_1,total_charges_sp_2,total_charges_sp_3,total_charges_sp_4,total_charges_sp_5,monthly_charges_q_100,total_charges_q_100,monthly_charges_robust,total_charges_robust,monthly_charges_bin,total_charges_bin
0,29.85,29.85,0.014583,0.335266,0.554993,0.095040,0.000118,0.0,0.041243,0.457057,0.459607,0.042093,1.762313e-12,0.0,0.232137,0.024832,-0.749192,-0.406994,0.0,0.0
1,56.95,1889.50,0.000116,0.094742,0.554677,0.335807,0.014658,0.0,0.004345,0.230314,0.596051,0.167842,1.447607e-03,0.0,0.389899,0.578927,-0.248499,0.141301,1.0,1.0


Применять по отдельности каждое преобразование не всегда удобно, особенно когда их очень много. Поэтому их объединяют в пайплайн или колоночные преобразования, которые можно визуализировать, запустить одной командой и сохранить. Последуйте этому примеру и объедините все энкодеры в одно преобразование. 

**Задание 3**

Напишите код, который объединит преобразования над числовыми колонками в ColumnTransformer, а над категориальными — в Pipeline, используя энкодеры из предыдущих заданий. Затем объедините два получившихся объекта класса одним колоночным преобразованием. После чего объедините ваш преобразованный набор данных с изначальным, а результат сохраните в переменную df. 

In [18]:
numeric_transformer = ColumnTransformer(
    transformers=[('spl', encoder_spl, num_columns),('q', encoder_q, num_columns), ('rb', encoder_rb, num_columns), ('pol', encoder_pol, num_columns), ('kbd', encoder_kbd, num_columns)] # ваш код здесь #
)

categorical_transformer = Pipeline(
	steps=[('encoder', encoder_oh)] # ваш код здесь #
)

preprocessor = ColumnTransformer(
	transformers=[('num', numeric_transformer, num_columns), ('cat', categorical_transformer, cat_columns)], n_jobs=-1 # ваш код здесь #
)

encoded_features = preprocessor.fit_transform(df) # ваш код здесь #

transformed_df = pd.DataFrame(encoded_features, columns=preprocessor.get_feature_names_out()) # ваш код здесь #

df = pd.concat([df, transformed_df], axis=1) # ваш код здесь #
df.head(2)

,id,customer_id,begin_date,end_date,type,paperless_billing,payment_method,monthly_charges,total_charges,internet_service,...,num__pol__total_charges^3,num__kbd__monthly_charges,num__kbd__total_charges,cat__type_One year,cat__type_Two year,cat__payment_method_Credit card (automatic),cat__payment_method_Electronic check,cat__payment_method_Mailed check,cat__internet_service_Fiber optic,cat__gender_Male
0,1,7590-VHVEG,2020-01-01,NaT,Month-to-month,Yes,Electronic check,29.85,29.85,DSL,...,2.659702e+04,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,2,5575-GNVDE,2017-04-01,NaT,One year,No,Mailed check,56.95,1889.50,DSL,...,6.745912e+09,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0


**Задание 4**

Чтобы визуализировать получившееся общее преобразование, просто посмотрите на значение переменной preprocessor.

In [19]:
preprocessor

,transformers,"[('num', ...), ('cat', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,-1
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True
,force_int_remainder_cols,'deprecated'
,n_knots,3
,degree,4
,knots,'uniform'


Объект preprocessor класса ColumnTransformer, объявленный в предыдущем задании, можно сохранить в MLflow, в директорию preprocessor — точно так же, как и модель. Для этого выполните следующий код в Jupyter Notebook:

In [22]:
os.environ["MLFLOW_S3_ENDPOINT_URL"] = 'https://storage.yandexcloud.net'
os.environ["AWS_ACCESS_KEY_ID"] = "YCAJE3Nlz8iDILW5VTYM1ihQB"
os.environ["AWS_SECRET_ACCESS_KEY"] = "YCPjvS7uwhvJpUj3bKm8X-IX4QAwBIVsvX61IL44"
os.environ['MLFLOW_ARTIFACT_URI'] = 'http://s3-student-mle-20250529-05fed48463'

TRACKING_SERVER_HOST = '127.0.0.1'
TRACKING_SERVER_PORT = 5000

mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")
mlflow.set_registry_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")

experiment_id = mlflow.get_experiment_by_name(EXPERIMENT_NAME).experiment_id

with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    run_id = run.info.run_id

    mlflow.sklearn.log_model(preprocessor, "column_transformer") 

2025/07/19 15:56:39 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
